In [112]:
class stacks:
    def __init__(self):
        self.stack = []
    #count is the number of variable used   
        self.count = 0
    
    # Push the element into the stack
    def push(self, x):        
        self.stack.append(str(x))
    # Pop the element from the stack
    def pop(self):
        if len(self.stack)>0:
            return self.stack.pop()
    # Add the top 2 elements in the stack, if less than 2 elements present create a variable and use it 
    def add(self):
        if len(self.stack)>=2:
            last = self.pop()
            s_last = self.pop()
            if last.isnumeric() and s_last.isnumeric():
                self.push(str(int(last) + int(s_last)))
            else:
                self.push("("+last+"+"+s_last+")")
        elif len(self.stack)==1:
            last = self.pop()
            self.push("(x"+str(self.count)+"+"+last+")")
            self.count= self.count + 1
        else:
            self.push("(x"+str(self.count)+"+"+"x"+str(self.count+1)+")")
            self.count+=2
    # Subtract the top 2 elements in the stack, if less than 2 elements present create a variable and use it 
    def sub(self):
        if len(self.stack)>=2:
            last = self.pop()
            s_last = self.pop()
            if last.isnumeric() and s_last.isnumeric():
                self.push(str(int(last) - int(s_last)))
            else:
                self.push("("+last +"-"+s_last+")")
        elif len(self.stack)==1:
            last = self.pop()
            self.push("(x"+str(self.count)+"-"+last+")")
            self.count+=1
        else:
            self.push("(x"+str(self.count)+"-"+"x"+str(self.count+1)+")")
            self.count+=2
    # Multiply the top 2 elements in the stack, if less than 2 elements present create a variable and use it 
    def mul(self):
        if len(self.stack)>=2:
            last = self.pop()
            s_last = self.pop()
            if last.isnumeric() and s_last.isnumeric():
                self.push(str(int(last) * int(s_last)))
            else:
                self.push("("+last +"*"+s_last+")")
        elif len(self.stack)==1:
            last = self.pop()
            self.push("(x"+str(self.count)+"*"+last+")")
            self.count+=1
        else:
            self.push("(x"+str(self.count)+"*"+"x"+str(self.count+1)+")")
            self.count+=2
            
    # Swap the 2 elements in the stack
    def swap():
        if(len(self.stack))>=2:
            last = self.pop()
            s_last = self.pop()
            self.push(last)
            self.push(s_last)
            
    # print the expression 
    def expression(self):
        return self.stack[0]
    

In [113]:
s = stacks()
s.push(1)
s.sub()
s.expression()

'(x0-1)'

In [115]:
#test 1
s = stacks()
s.push(2)
s.push(3)
s.mul()
s.push(1)
s.add()
s.expression()

'7'

In [114]:
# test 2
s = stacks()
s.push(2)
s.mul()
s.push(1)
s.mul()
s.expression()

'(1*(x0*2))'

In [111]:
#taking input from the console
print("Enter the number of operations:")
n = int(input())
s = stacks()
while n!=0:
    print("Enter operation name:")
    op = input()
    if op == "PUSH":
        print("Enter value to be pushed")
        val = (int(input()))
        s.push(val)
    elif op == "ADD":
        s.add()
    elif op == "SUB":
        s.sub()
    elif op == "MUL":
        s.mul()
    elif op == "SWAP":
        s.swap()
    n -=1

# print the output
s.expression()

Enter the number of operations:
4
Enter operation name:
PUSH
Enter value to be pushed
10
Enter operation name:
SUB
Enter operation name:
MUL
Enter operation name:
ADD


'(x2+(x1*(x0-10)))'